In [ ]:
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
import pickle

In [ ]:
import sys
sys.path.insert(0, '..')

%run plate_EQS.ipynb


In [ ]:
import Pinn_Worker

In [ ]:
valData = [['u.txt'],'ofen']
test_gridObj = SEQ.Grid(3,{'x':0,'y':1,'t':2},[[[-2,2],[-2,2],[0,10]]],10)
configspecs = {
    'denspt':[3,8],
    'numNeurons':[10,100],
    'numLayers':[2,50],
    'activator':['tanh','relu','softmax','sigmoid'],
    'loss':['mae','mse'],
    'optimizer':['Adam','RMSprop','SGD','Nadam','Ftrl'],
    'batch_size':[5000,10000,15000,25000]
}




In [ ]:
# Step 1: Start a nameserver
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with the default port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
# Note the run_id argument. This uniquely identifies a run of any HpBandSter optimizer.
NS = hpns.NameServer(run_id='example1', host='127.0.0.1', port=None)
NS.start()

# Step 2: Start a worker
# Now we can instantiate a worker, providing the mandatory information
# Besides the sleep_interval, we need to define the nameserver information and
# the same run_id as above. After that, we can start the worker in the background,
# where it will wait for incoming configurations to evaluate.
numWorkers = 6
experiment_name = 'plate_exp'
run_id='plate_Gridsearch_1'


workers=[]
for i in range(1,numWorkers+1):
    print('Creating worker')
    w = Pinn_Worker.PINN_Worker(
    valData = valData,
    test_gridObj=test_gridObj,
    PDESystem=mySys,
    configspecs = configspecs,
    valFromFEM=True,
    experiment_name = experiment_name,
    nameserver='127.0.0.1',
    run_id=run_id,
    id=i)
    
    print('Worker created, starting it')
    w.run(background=True)
    print('Worker running, adding it to the list')
    workers.append(w)
    print('Worker added')

In [ ]:
bohb = BOHB(  configspace = w.get_configspace(),
              run_id = run_id, nameserver='127.0.0.1',
              min_budget=10, max_budget=100
           )
res = bohb.run(n_iterations=10
              )


In [ ]:
np.save('HpBandster_Results',res,allow_pickle=True)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()